In [26]:
from mediawikiapi import MediaWikiAPI
mediawikiapi = MediaWikiAPI()
page = mediawikiapi.page("ad astra (film)")
content = page.content
print(content)

# get the content between '== Cast ==' and next topic
content = content.split('== Cast ==')[1].split('==')[0]
print(content)

Ad Astra is a 2019 American psychological science fiction film produced, co-written, and directed by James Gray. Starring Brad Pitt (who also produced), Tommy Lee Jones, Ruth Negga, Liv Tyler, and Donald Sutherland, it follows an astronaut who ventures into space in search of his lost father, whose obsessive quest to discover intelligent alien life at all costs threatens the Solar System and all life on Earth. The project was announced in early 2016, with Gray saying he wanted to feature "the most realistic depiction of space travel that's been put in a movie". Pitt signed on to star in April 2017 and the rest of the cast joined later that year. Filming began around Los Angeles that August, lasting through October.
Ad Astra premiered at the Venice Film Festival on August 29, 2019, and was theatrically released in the United States on September 20, 2019, by Walt Disney Studios Motion Pictures via 20th Century Fox. It received positive reviews from critics, with praise for Pitt's perform

In [39]:
import requests
import json
import pandas as pd
import sqlite3

# url encoding library
import urllib.parse

api_key = 'c84d2eacda018a9f013d10a6880631ac'

# CONNECT TO movie_corpus.db
conn = sqlite3.connect('movie_corpus.db')
cursor = conn.cursor()

# GET ALL CHARACTERS GROUPED BY MOVIE
cursor.execute('SELECT name, title FROM character JOIN movie WHERE movie.movie_id = character.movie_id GROUP BY title')

# convert to url encoding
title = urllib.parse.quote_plus(title)

print(title)

url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}'

resp = requests.get(url).json()

print(resp)

id = resp['results'][0]['id']

url = f'https://api.themoviedb.org/3/movie/{id}?api_key={api_key}&append_to_response=credits'

resp = requests.get(url).json()

#Print readable JSON
print(json.dumps(resp, indent=4, sort_keys=True))

next+friday
{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/xi4Uy11TdACds3SXglrbbmaVd90.jpg', 'genre_ids': [35], 'id': 10471, 'original_language': 'en', 'original_title': 'Next Friday', 'overview': 'A streetwise man flees South Central Los Angeles, heading to the suburbs and his lottery-winner uncle and cousin, to avoid a neighborhood thug with a grudge who has just escaped from prison.', 'popularity': 13.944, 'poster_path': '/dP4fYCGQZzg17ta7FvLwJVCwwt9.jpg', 'release_date': '2000-01-12', 'title': 'Next Friday', 'video': False, 'vote_average': 6.434, 'vote_count': 509}, {'adult': False, 'backdrop_path': '/tPltOE6HiKoIrbgXYIr7mrUaCEN.jpg', 'genre_ids': [35], 'id': 10426, 'original_language': 'en', 'original_title': 'Friday After Next', 'overview': "Craig and his cousin Day Day have finally moved out of their parents' houses and into their own crib, working nights at a local mall as security guards. When their house is robbed on Christmas Eve they set out to track down the c

In [7]:
import requests
import json
import pandas as pd
import sqlite3

# url encoding library
import urllib.parse

api_key = 'c84d2eacda018a9f013d10a6880631ac'

# CONNECT TO movie_corpus.db
conn = sqlite3.connect('final.db')
cursor = conn.cursor()

# GET ALL CHARACTERS GROUPED BY MOVIE
characterFilm = cursor.execute('SELECT name, title FROM character JOIN movie WHERE movie.id = character.movie_id ORDER BY title').fetchall()

titles = cursor.execute('SELECT title FROM movie ORDER BY title').fetchall()

# CREATE MOVIE - CHARACTERS DICTIONARY
movieCharactersDatabase = {}
movieCharactersAPI = {}
movieActorsAPI = {}



In [11]:
for title in titles:
    
    title = title[0]
    print(f'Title - {title}')
    
    movieCharactersDatabase[title] = []
    movieCharactersAPI[title] = []
    movieActorsAPI[title] = []
    
    # convert to url encoding
    titleParsed = urllib.parse.quote_plus(title)
    
    print(f'TitleParsed - {titleParsed}')
    
    url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={titleParsed}'
    resp = requests.get(url).json()
    id = resp['results'][0]['id']
    url = f'https://api.themoviedb.org/3/movie/{id}?api_key={api_key}&append_to_response=credits'
    resp = requests.get(url).json()
    
    # GET CHARACTERS FROM CAST
    cast = resp['credits']['cast']
    for character in cast:
        movieCharactersAPI[title].append(character['character'])
        movieActorsAPI[title].append(character['name'])

Title - 10 things i hate about you
TitleParsed - 10+things+i+hate+about+you
Title - 1492: conquest of paradise
TitleParsed - 1492%3A+conquest+of+paradise
Title - 15 minutes
TitleParsed - 15+minutes
Title - 2001: a space odyssey
TitleParsed - 2001%3A+a+space+odyssey
Title - 48 hrs.
TitleParsed - 48+hrs.
Title - 8mm
TitleParsed - 8mm
Title - a bucket of blood
TitleParsed - a+bucket+of+blood
Title - a clockwork orange
TitleParsed - a+clockwork+orange
Title - a hard day's night
TitleParsed - a+hard+day%27s+night
Title - a nightmare on elm street
TitleParsed - a+nightmare+on+elm+street
Title - a nightmare on elm street 3: dream warriors
TitleParsed - a+nightmare+on+elm+street+3%3A+dream+warriors
Title - a nightmare on elm street 4: the dream master
TitleParsed - a+nightmare+on+elm+street+4%3A+the+dream+master
Title - a nightmare on elm street part 2: freddy's revenge
TitleParsed - a+nightmare+on+elm+street+part+2%3A+freddy%27s+revenge
Title - a nightmare on elm street: the dream child
Title

In [3]:
# ITERATE THROUGH ALL MOVIES
for character in characterFilm:
     
    # GET MOVIE TITLE
    title = character[1]
    
    # GET CHARACTERS
    movieCharactersDatabase[title].append(character[0])
    
# ITERATE THROUGH ALL movieCharactersDatabase

In [5]:

from thefuzz import fuzz
from thefuzz import process
countEmptyString = 0

for title in movieCharactersDatabase:
    # GET CHARACTERS
    charactersDatabase = movieCharactersDatabase[title]
    charactersAPI = movieCharactersAPI[title]
    
    for character in charactersDatabase:
        

        if character == None:
            print(f'{title} - {character}, {characterAPI}')
        character = character.lower().replace('.', '').replace('-', '')
        
        maxFuzzRatio = 0
        bestCharAPI = ''
        
        for characterAPI in charactersAPI:
            
            characterAPICopy = characterAPI
            characterAPI = characterAPI.lower().replace('.', '').replace('-', '')
                        
            if characterAPI == '':
                continue
            
            fuzzRatio = fuzz.ratio(character, characterAPI)
            
            if fuzzRatio > 90 or character in characterAPI or characterAPI in character:
                
                if fuzzRatio > maxFuzzRatio:
                    maxFuzzRatio = fuzzRatio
                    bestCharAPI = characterAPICopy
        
        if bestCharAPI == '':
            countEmptyString += 1
       
        print(f'{title} - {character}, {bestCharAPI}')
        print('===========')
        
print(f'Empty Strings - {countEmptyString}')

            

                

10 things i hate about you - bianca, Bianca Stratford
10 things i hate about you - bruce, 
10 things i hate about you - sharon, 
10 things i hate about you - walter, Walter Stratford
10 things i hate about you - cameron, Cameron James
10 things i hate about you - chastity, Chastity
10 things i hate about you - joey, Joey Donner
10 things i hate about you - kat, Katarina Stratford
10 things i hate about you - mandella, Mandella
10 things i hate about you - michael, Michael Eckman
10 things i hate about you - miss perky, 
10 things i hate about you - patrick, Patrick Verona
1492: conquest of paradise - alonso, Alonso de Bolaños
1492: conquest of paradise - arojaz, 
1492: conquest of paradise - beatrix, Beatrix
1492: conquest of paradise - bobadilla, Bobadilla
1492: conquest of paradise - columbus, Christopher Columbus
1492: conquest of paradise - fernando, Older Fernando
1492: conquest of paradise - isabel, Queen Isabella
1492: conquest of paradise - marchena, Marchena
1492: conquest of 

In [4]:
!pip install thefuzz

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for thefuzz from https://files.pythonhosted.org/packages/19/7d/ca50835332895beb87e663f9a610a7e0a7335b69e31177aee87acc3db9bd/thefuzz-0.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for rapidfuzz<4.0.0,>=3.0.0 from https://files.pythonhosted.org/packages/8e/f4/e99caa0ea652e79bb209bf53ec1be5f34ecd3db5f874ccf41fb7d5285e6a/rapidfuzz-3.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.5 MB/s eta 0:00:0000:0100:01
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
